Rusty Bargain used car sales service is developing an app to attract new customers. In that app, you can quickly find out the market value of your car. You have access to historical data: technical specifications, trim versions, and prices. You need to build the model to determine the value.

Rusty Bargain is interested in:

- the quality of the prediction;
- the speed of the prediction;
- the time required for training

In [52]:
!pip install optuna
!pip install catboost

In [53]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb
import optuna

from google.colab import drive
drive.mount('/content/drive')

## Data preparation

In [54]:
path = "/content/drive/My Drive/Colab Notebooks/Practicum/car_data.csv"
df = pd.read_csv(path)
df.info()

In [55]:
df.head()

### Data Cleaning

#### Converting data type

Transform data type strategy:

* Int: `Price`, `Power`, `Mileage`, `PostalCode`, `NumberOfPictures`
* Datetime: `DateCrawled`, `RegistrationYear`, `RegistrationMonth`, `DateCreated`, `LastSeen`
* String: `VehicleType`, `Gearbox`, `Model`, `FuelType`, `Brand`, `NotRepaired`

Most of string type columns can be transformed to categorical data. I'll do it later.

##### Datetime

Before converting the datetime type, I'll drop the column of `DateCrawled`, it seems redundant and useless for training.

In [56]:
df.drop('DateCrawled', axis=1, inplace=True)

###### `RegistrationYear`

During converting the 'RegistrationYear' column, I found there are abnormal years

In [57]:
df['RegistrationYear'].unique()

In [58]:
registration_mask = (df['RegistrationYear'] < 1900) | (df['RegistrationYear'] > 2016)
df[registration_mask]['RegistrationYear'].nunique()

Delete abnormal values

In [59]:
df = df[~registration_mask]

There are two columns 'RegistrationYear' and 'RegistrationMonth', but with the same meaning that to indicate the registration time of the car. I'll combine them together and convert it to datetime.

###### `RegistrationMonth`

In [60]:
df['RegistrationMonth'].unique()

In [61]:
df['RegistrationMonth'].value_counts()

There are 13 unique month values, from 0 to 12, in the column. In `pandas.Series.dt.month`, the correct month values are from 1 to 12, therefore the value 0 is abnormal.

Sicne the count number of 0 is 37220, too many to be deleted. I'll break down the values in month of 0 and randomly allocate them to other month groups

In [62]:
month_mask = df['RegistrationMonth'] == 0
random_months = np.random.randint(1, 13, size=month_mask.sum())
df.loc[month_mask, 'RegistrationMonth'] = random_months
df['RegistrationMonth'].unique()

In [63]:
df['RegistrationMonth'].value_counts()

Combine 'RegistrationYear' and 'RegistrationMonth' and convert to 'datetime'. Then only keep the `RegistrationDate` as the time information column, drop others

In [64]:
#df['RegistrationDate'] = pd.to_datetime(df['RegistrationYear'] * 10000 + df['RegistrationMonth'] * 100 + 1, format='%Y%m%d').dt.date
df['RegistrationDate'] = pd.to_datetime(df['RegistrationYear'] * 10000 + df['RegistrationMonth'] * 100 + 1, format='%Y%m%d')
# df['DateCreated'] = pd.to_datetime(df['DateCreated']).dt.date.astype('datetime64[ns]')
df['LastSeen'] = pd.to_datetime(df['LastSeen'])
df.drop(['RegistrationYear', 'RegistrationMonth', 'DateCreated', 'LastSeen'], axis=1, inplace=True)
df.head()

#### Missing values

In [65]:
df.isna().sum()

There are lots of missing values in 5 columns, and cannot be deleted since the missing values take the significant proportion.

It's not hard to fill some missing values rely on other information, such as I could fill the `VehicleType` and `FuelType` by the information of `Model`. However, I'm not sure whether it will reflect the prediction result, for example, when customers are browsing the used car website, they would tend to trust the cars with fully information and ignore the cars with certain blank terms, therefore the cars lack of `Notrepaired` information will less likely be trusted and sold. For this reason, If I fill the missing values by my logistic inference, I might affect the prediction result subjectively.

So I will fill the missing values with a simple 'Unknown'.

In [66]:
df['VehicleType'] = df['VehicleType'].fillna('Unknown')
df['Gearbox'] = df['Gearbox'].fillna('Unknown')
df['Model'] = df['Model'].fillna('Unknown')
df['FuelType'] = df['FuelType'].fillna('Unknown')
df['NotRepaired'] = df['NotRepaired'].fillna('Unknown')
df.info()

#### Check duplicates

In [67]:
df.duplicated().sum()

In [68]:
df = df.drop_duplicates()
df.duplicated().sum()

#### Check abnormal values

In [69]:
df.describe()

There are abnormal values in all numeric columns:

* `Price`: The minimum value is 0, it's abnormal.
* `Power`: The horsepower of a car changes by the vehicle displacement. It's between tens and hundreds. So the minimum and maximum values are abnormal.
* `Mileage`: Half of entries are the same value which is 150000.
* `NumberOfPictures`: All entries equal to 0. This column is useless
* `PostalCode`: Some entries have 4 digits, while most of the entries have 5 digits.

I don't have any other information to deal with `Mileage`, it's too many to be removed directly.

For `PostalCode`, I don't know whether there is any wrong values either. Fortunately this numbers are categorical, the value of number is doesn't matter.

I will address 3 columns, `Price`, `Power` and `NumberOfPictures`. I'll use the abnormal detection theory in statistics to calculate the threshold of normal values, i.e. [Q1 - (1.5 * IQR), Q3 + (1.5 * IQR)]. For `NumberOfPictures`, I'll just remove it.









In [70]:
def cal_thresholds(data_array):
    q1 = np.percentile(data_array, 25)
    q3 = np.percentile(data_array, 75)
    iqr = q3 - q1
    lower_threshold = q1 - 1.5 * iqr
    upper_threshold = q3 + 1.5 * iqr
    return lower_threshold, upper_threshold

lower_price, _ = cal_thresholds(df['Price'])
lower_power, upper_power = cal_thresholds(df['Power'])

print(lower_price, lower_power, upper_power)

The `lower_price` and `lower_power` are negative, don't make sense. I'll set the threshold manually.

In [71]:
mask_price = df['Price'] > 100
mask_power = (df['Power'] > 40) & (df['Power'] < upper_power)
df = df[mask_price & mask_power]
df.drop('NumberOfPictures', axis=1, inplace=True)

In [72]:
df.describe()

## Model training

After data cleaning, the next step should be Feature Engineering which mainly to encode the categorical columns. Since I'll try different models, i.e. linear regression models, tree based models and gradient boost models, therefore I'll apply different encoding methods to fit the requests of training models

| Model | Encoding requirements | Accept datetime format| Need scaling |
| --    |   -- | -- | -- |
| Linear Regression | One-hot Encoding, Features Hashing| No| Yes |
| Decision Tree | Label Encoding | Yes | No |
| Random Forest | Label Encoding| Yes | No |
| LightGBM | No specific requirement| Yes | No |
| CatBoost | No specific requirement | Yes | No|
| XGBoost | One-hot Encoding | Yes| No |

In [ ]:
categorical = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
for column in categorical:
    print(df[column].nunique())

### Linear Regression, XGBoost with One-hot Encoding

##### One-hot encoding

The features of `Model` and `Brand` has tens to hundreds unique values, they belong to high cardinality variables, the one-hot encoding is not suitable for them. Other features can be applied the one-hot encoding.

In [ ]:
categorical = ['VehicleType', 'Gearbox',  'FuelType', 'NotRepaired']
encoded_df = pd.get_dummies(df, columns=categorical)
encoded_df.reset_index(drop=True, inplace=True)

In [ ]:
encoded_df.shape

##### features hashing

For high caridinality variables, I'll apply the Feature Hash Encoding to convert the categorical data to numeric

The number of unique values of `model` is more than 250, I'll set `n_features=500`

In [ ]:
def feature_hasher(df, column, n_features, prefix):
    hasher = FeatureHasher(n_features=n_features, input_type='string')
    hashed_features = hasher.transform(df[column].apply(lambda x: list(x)))
    hashed_df = pd.DataFrame(hashed_features.toarray())
    hashed_df = hashed_df.add_suffix('_' + prefix)
    return hashed_df

hashed_model_df = feature_hasher(encoded_df, 'Model', 256, 'hashed_model')
hashed_brand_df = feature_hasher(encoded_df, 'Brand', 64, 'hashed_brand')
encoded_df = pd.concat([encoded_df, hashed_model_df, hashed_brand_df], axis=1) #, hashed_brand_df
drop_list = ['Model', 'Brand']
encoded_df.drop(drop_list, axis=1, inplace=True)
ohe_df = encoded_df.copy() # for XGBoost

#### Convert datetime to float

I need to convert datetime to float in order to feed to Linear Regression model.

In [ ]:
# Convert the datetime to float, otherwise it'll be error during training with LR model
encoded_df["RegistrationDate"] = encoded_df['RegistrationDate'].values.astype(float) / 1e9 / 86400 # only keep year, month and day information, get rid of nanoseconds



```
# This is formatted as code
```

#### Train pipeline

##### Prepare training and testing set



In [ ]:
def train_test_prepare(df, target, test_size):
    X = df.drop(target, axis=1)
    y = df[target]
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    return X_train, y_train, X_val, y_val, X_test, y_test

##### Standard scaling

In [ ]:
def standard_scaling(X_train, X_val):
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train.loc[:, :] = scaler.transform(X_train.loc[:, :])
    X_val.loc[:, :] = scaler.transform(X_val.loc[:, :])

    return X_train, X_val

##### Training and evaluate

In [ ]:
model_list = []
training_time_list = []
predict_time_list = []
rmse_list = []

In [ ]:
def train_pipeline(X_train,X_val, y_train, y_val, model_name, model_obj, scaling=False, lightGBM_params={}):
    start = time.time()
    if scaling is True:
        X_train, X_val = standard_scaling(X_train, X_val)
    if model_name == 'LightGBM' or model_name == 'Optimized LightGBM':
        train_data = lgb.Dataset(X_train, label=y_train)
        model = lgb.train(lightGBM_params, train_data)
    elif model_name == 'CatBoost' or model_name == 'Optimized CatBoost':
        model = model_obj
        model.fit(X_train, y_train, verbose=250)
    else:
        model = model_obj
        model.fit(X_train, y_train)
    time_cost = round((time.time() - start), 3)
    training_time_list.append(time_cost) # seperate training and predict time
    predict_start = time.time()
    y_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False) # squared: If True returns MSE value, if False returns RMSE value.
    time_cost = round((time.time() - predict_start), 3)
    print(f"The RMSE of {model_name} with default setting is: {rmse}")
    model_list.append(model_name)
    predict_time_list.append(time_cost)
    rmse_list.append(rmse)

Linear Regression

In [ ]:
X_train_ohe, y_train_ohe, X_val_ohe, y_val_ohe, X_test_ohe, y_test_ohe = train_test_prepare(encoded_df, 'Price', 0.3)
train_pipeline(X_train_ohe, X_val_ohe, y_train_ohe, y_val_ohe, "Linear Regression", LinearRegression(), scaling=True)

XGBoost

In [ ]:
train_pipeline(X_train_ohe, X_val_ohe, y_train_ohe, y_val_ohe, "XGBoost with default setting", xgb.XGBRegressor(use_label_encoder=False, random_state=42))

Delete the encoded data to free the memory.

In [ ]:
del encoded_df

### Random Forest, LightGBM, CatBoost with label Encoding

#### Label Encoding

In [73]:
categorical = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
label_encoder = OrdinalEncoder()
label_encoded_data = label_encoder.fit_transform(df[categorical])
reset_index_df = df.reset_index(drop=True).copy()
label_encoded_df = pd.concat([reset_index_df.drop(columns=categorical), pd.DataFrame(label_encoded_data, columns=categorical)], axis=1)
label_encoded_df.head()

In [74]:
label_encoded_df["RegistrationDate"] = pd.to_datetime(label_encoded_df['RegistrationDate']).values.astype(float)

Random Forest with default setting

In [75]:
X_train_label, y_train_label, X_val_label, y_val_label, X_test_label, y_test_label = train_test_prepare(label_encoded_df, 'Price', 0.3)
train_pipeline(X_train_label, X_val_label, y_train_label, y_val_label, "Random Forest", RandomForestRegressor())

Fine tuning with random search method

Note: The random search process will take more than 1 hour, I'll comment this code and assign the learned parameter directly.

In [ ]:
def random_optimize(estimator, param_grid, X_train, y_train, n_iter, cv):
    random_search = RandomizedSearchCV(estimator=estimator, param_distributions=param_grid, n_iter=n_iter, cv=cv)
    random_search.fit(X_train, y_train)
    print("Best parameters: ", random_search.best_params_)
    print("Best score: ", random_search.best_score_)
    return random_search.best_params_

In [ ]:
# param_grid_rfr = {
#     'n_estimators': [100, 200],
#     'max_depth': [None, 5, 10],
#     'min_samples_split': [2, 4, 8],
#     'min_samples_leaf': [1, 2, 4, 8]
# }
# best_params = random_optimize(RandomForestRegressor(random_state=42), param_grid_rfr, X_train_label, y_train_label, n_iter=10, cv=5)

In [ ]:
#best_params = random_search_rfr.best_params_
best_params = {'n_estimators': 200, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_depth': None}
train_pipeline(X_train_label, X_val_label, y_train_label, y_val_label, "Random Forest with optimized parameters", RandomForestRegressor(random_state=42, **best_params))

After fine tuning, I got a slightly better result.

LightGBM with default setting

According to the LightGBM document, it can use categorical features directly. But they should be convert to numeric data. So I'll use the dataset which was label encoded for the tree based model.

In [ ]:
train_pipeline(X_train_label, X_val_label, y_train_label, y_val_label, "LightGBM", None)

Fine tuning. I'll use the Optuna, a popular hyperparameter optimization framework, to search the best parameters.

In [ ]:
def objective(trial):
    # Define hyperparameter search space
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
    }
    train_data = lgb.Dataset(X_train_label, label=y_train_label)
    model = lgb.train(params, train_data)
    y_pred_label = model.predict(X_val_label)
    rmse = mean_squared_error(y_val_label, y_pred_label)

    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
best_params_lightGBM = study.best_params
print("Best Hyperparameters:", best_params_lightGBM)

In [ ]:
train_pipeline(X_train_label, X_val_label, y_train_label, y_val_label, "Optimized LightGBM", None, lightGBM_params=best_params_lightGBM)

The RMSE has optimized 5.5% after 10 trials.

### CatBoost

#### Default settings

In [ ]:
train_pipeline(X_train_label, X_val_label, y_train_label, y_val_label, "CatBoost", CatBoostRegressor())

I'll still use `optuna` to fine tune the model

In [ ]:
def objective(trial):
    # Define hyperparameter search space
    params = {
        'iterations': 100,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 100),
    }

    # Create and train the CatBoost model with current hyperparameters
    model = CatBoostRegressor(**params)
    model.fit(X_train_label, y_train_label, verbose=False)

    # Make predictions on the test set
    y_pred_label = model.predict(X_val_label)

    # Calculate the Mean Squared Error (MSE)
    mse = mean_squared_error(y_val_label, y_pred_label)

    return mse

# Optimize hyperparameters using Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Get the best hyperparameters
best_params_cat = study.best_params
print("Best Hyperparameters:", best_params_cat)

In [ ]:
train_pipeline(X_train_label, X_val_label, y_train_label, y_val_label, "Optimized CatBoost", CatBoostRegressor(**best_params_cat))

## Model analysis

In [ ]:
results = pd.DataFrame({'Model': model_list, 'RMSE': rmse_list, 'Training Time Cost (second)': training_time_list, 'Predict Time Cost (second)': predict_time_list})
results

Based on the "results" table, we can analyze the different models from both the quality and speed perspectives:

* Quality Analysis:

1. **Linear Regression**: The model has the highest Root Mean Squared Error (RMSE) of 2638.29, indicating that its predictive accuracy is sanity check.
2. Other models except the `Optimized CatBoost` obtain the metrics between 1500 to 1600. The worst among them is `LightGBM` with RMSE is 1591.93, the best among them is `CatBoost` with RMSE is 1516.50.
3. **Optimized CatBoost**: The optimized CatBoost model further improves its performance, achieving an RMSE of 1472.95, making it the best-performing model among all the listed models.

* Speed Analysis:

    I seperated the prediction time out from the training time.

    For training time, there are 3 hierarchies. `XGBoost` and the Random Forest models take the longest training duration. `Linear Regression` and the CatBoost models lie in the middle with tens seconds. LightGBM models are super fast, it only takes 2 - 3 seconds to train 3000000 entries data.

    For prediction time, all models except Random Forest only take less than 1 second. The fastest model is `CatBoost` with default settings, it only takes 0.06 second. The `Optimized CatBoost` and `Linear Regression` are slightly slower with 0.177 second. `XGBoost` and LightGBM take 0.2 - 0.4 second.

In summary, the `CatBoost` and `Optimized CatBoost` model achieves both the best quality and speed among all models. `CatBoost` takes 1/3 prediction time of `Optimized CatBoost`, while its RMSE is 3% worse than `Optimized CatBoost`. Alternatively, if a computation time of less than 0.2 second is acceptable, then the `Optimized CatBoost` can be chosen as it offers the highest accuracy.

### Evaluate final model on test set

In [76]:
model = CatBoostRegressor()
model.fit(X_train_label, y_train_label, verbose=250)
y_pred = model.predict(X_test_label)
rmse = mean_squared_error(y_test_label, y_pred, squared=False) # squared: If True returns MSE value, if False returns RMSE value.
print(f"The RMSE of final model with fine-tuning setting is: {rmse}")

Finally, I applied the `CatBoost` with default settings on the test set and obtain the RMSE of 1520.68

> Updated

1. I've used the training, validation and test sets for training and ev

# Checklist

Type 'x' to check. Then press Shift+Enter.

- [x]  Jupyter Notebook is open
- [x]  Code is error free
- [x]  The cells with the code have been arranged in order of execution
- [x]  The data has been downloaded and prepared
- [x]  The models have been trained
- [x]  The analysis of speed and quality of the models has been performed